# Reseau de neuronne

## Importation des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

2023-01-19 20:17:32.670567: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 20:17:32.941290: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2023-01-19 20:17:32.941348: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-19 20:17:34.660975: W tensorflow/compiler/xl

## I. Lecture des descripteurs

In [2]:
data = pd.read_csv('../../inference/dataset.csv')
data = np.array(data.values)
label_names = data[:,1024]
unique_label = np.unique(label_names)
feature_values = data[:, :1024]

## II. Réseau de neuronne

On commence par séparer les données en deux ensembles: un pour l'apprentissage et l'autre pour l'évaluation du modèle

In [3]:
# split données
X_train, X_test, y_train, y_test = train_test_split(feature_values, label_names, test_size=0.33, random_state=43)

On poursuit en transformant les labels en indice:

In [4]:
# enc labelNames to indices
encName2Ind = preprocessing.LabelEncoder()
encName2Ind.fit(unique_label)
labelIndices_unique = encName2Ind.transform(unique_label)
labelIndices  = encName2Ind.transform(label_names)

# Conversion des noms des labels en indices
labelInd_train = encName2Ind.transform(y_train)
labelInd_test = encName2Ind.transform(y_test)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(labelInd_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(labelInd_test, dtype=tf.float32)

2023-01-19 20:17:38.230246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2023-01-19 20:17:38.230300: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-19 20:17:38.230352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (nicolas-Lenovo-IdeaPad-C340-14IWL): /proc/driver/nvidia/version does not exist
2023-01-19 20:17:38.231090: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

On normalise les données pour réduire la complexité et donner le même poids à tous les descripteurs.

In [5]:
# Normalisation 
images_mean = np.mean(X_train, axis=0, keepdims=True)
images_std = np.std(X_test, axis=0, keepdims=True) + 1e-7
X_train = (X_train - images_mean) / images_std
X_test = (X_test - images_mean) / images_std

/home/nicolas/Documents/3A_ENSTA/ml_embedded/vultr-tensorflow/lib/python3.8/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


On crée ensuite le réseau de neuronne.

In [6]:
## creation du reseau de neuronne
model = Sequential(name="Sequential_CNN")
# couche de classification
model.add(Dense(256, activation='relu', input_shape = [1024]))
model.add(Dense(len(unique_label), activation='softmax'))
model.summary()

Model: "Sequential_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 264,970
Trainable params: 264,970
Non-trainable params: 0
_________________________________________________________________


Apprentissage :

In [7]:
# paramètres
NB_EPOCHS = 40
LEARNING_RATE = 0.1
MOMENTUM = 0.09
DECAY = 0.01

#  Choix de la tâche à effectuer = choix de la fonction de loss/pertes/coûts à minimise
loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy()# sans alias

# compilation ...
model.compile(loss=loss_fcn, optimizer='ADAM', metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_asbolute_error', 'mean_squared_error'])

#... et entrainement
history = model.fit(X_train, y_train, epochs=NB_EPOCHS)

Epoch 1/40
21/21 [==============================] - 1s 7ms/step - loss: 1.9947 - accuracy: 0.3034
Epoch 2/40
21/21 [==============================] - 0s 7ms/step - loss: 1.5625 - accuracy: 0.4559
Epoch 3/40
21/21 [==============================] - 0s 5ms/step - loss: 1.3477 - accuracy: 0.5411
Epoch 4/40
21/21 [==============================] - 0s 6ms/step - loss: 1.2150 - accuracy: 0.5859
Epoch 5/40
21/21 [==============================] - 0s 6ms/step - loss: 1.1087 - accuracy: 0.6472
Epoch 6/40
21/21 [==============================] - 0s 6ms/step - loss: 1.0326 - accuracy: 0.6667
Epoch 7/40
21/21 [==============================] - 0s 6ms/step - loss: 0.9950 - accuracy: 0.6487
Epoch 8/40
21/21 [==============================] - 0s 6ms/step - loss: 0.9339 - accuracy: 0.7025
Epoch 9/40
21/21 [==============================] - 0s 6ms/step - loss: 0.9008 - accuracy: 0.7294
Epoch 10/40
21/21 [==============================] - 0s 7ms/step - loss: 0.8183 - accuracy: 0.7623
Epoch 11/40
21/21 [

## III. Evaluation du modèle

In [8]:
#Confution Matrix and Classification Report
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
print()
print()
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))
print()
print()
print('Classification Report')
print(classification_report(labelInd_test, y_pred, target_names=unique_label))

11/11 [==============================] - 0s 2ms/step


Confusion Matrix
[[16  0  1  2  1  0  1  0  2  7]
 [ 0 28  1  0  0  1  0  0  0  0]
 [ 2  1 17  2  4  1  0  0  2  8]
 [ 2  1  0 25  0  0  5  2  1  6]
 [ 0  0  3  2 25  0  0  3  2  1]
 [ 1 13  0  0  0 11  1  0  0  4]
 [ 2  0  0  0  1  0 26  0  0  3]
 [ 0  1  0  1  0  1  0 29  0  0]
 [ 1  1  2  4  4  0  2  2 17  1]
 [ 1  3  1  3  0  1  5  0  2 11]]


Classification Report
              precision    recall  f1-score   support

       blues       0.64      0.53      0.58        30
   classical       0.58      0.93      0.72        30
     country       0.68      0.46      0.55        37
       disco       0.64      0.60      0.62        42
      hiphop       0.71      0.69      0.70        36
        jazz       0.73      0.37      0.49        30
       metal       0.65      0.81      0.72        32
         pop       0.81      0.91      0.85        32
      reggae       0.65      0.50      0.57        34
        rock       0.27      0.4

## IV. Enregistrement de l'ensemble d'évaluation

In [9]:
txt = ""
txt += "#ifndef TEST_DATASET_H\n#define TEST_DATASET_H\n\n"
txt += "#define N_TEST_EXAMPLES " + str(X_test.shape[0]) + "\n"
txt += "#define N_FEATURES " + str(X_test.shape[1]) + "\n"
txt += "#define N_CLASSES 10\n\n"
print(model.evaluate(X_test, y_test))
txt += "float test_acc=" + str(model.evaluate(X_test, y_test)[1]) +";\n\n"

txt += "float X_test[N_TEST_EXAMPLES][N_FEATURES] = {\n"
for i in range(X_test.shape[0]-1):
    txt += "{"
    for j in range(X_test.shape[1]-1):
        txt += str(X_test[i,j].numpy()) +","
    txt += str(X_test[i,X_test.shape[1]-1].numpy()) + "},\n"
txt += "{"
for j in range(X_test.shape[1]-1):
    txt += str(X_test[X_test.shape[0]-1,j].numpy()) +","
txt += str(X_test[X_test.shape[0]-1,X_test.shape[1]-1].numpy()) + "}};\n\n"

txt += "float y_test[N_FEATURES]={"
for i in range(X_test.shape[0]-1):
    txt += str(y_test[i].numpy()) +","
txt += str(y_test[X_test.shape[0]-1].numpy()) +"};\n\n"

txt += "#endif"

# écriture du code dans le fichier
fichier = open("../../inference/NeuronalNetwork/test_dataset.h", "w")
fichier.write(txt)
fichier.close()

11/11 [==============================] - 0s 3ms/step - loss: 1.9913 - accuracy: 0.6212
[1.9912750720977783, 0.6212121248245239]
11/11 [==============================] - 0s 3ms/step - loss: 1.9913 - accuracy: 0.6212


## V. Enregistrement du modèle sous tensorflow-lite

In [10]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('../../inference/NeuronalNetwork/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmporgan2e4/assets


INFO:tensorflow:Assets written to: /tmp/tmporgan2e4/assets
2023-01-19 20:19:10.644433: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-19 20:19:10.644473: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-19 20:19:10.645185: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmporgan2e4
2023-01-19 20:19:10.646500: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-19 20:19:10.646549: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmporgan2e4
2023-01-19 20:19:10.649736: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-01-19 20:19:10.650574: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-19 20:19:10.687657: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

## VI. Enregistrement optimisé du modèle sous tensorflowLite

In [11]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open('../../inference/NeuronalNetwork/modelOptimize.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmph5nsxh41/assets


INFO:tensorflow:Assets written to: /tmp/tmph5nsxh41/assets
2023-01-19 20:19:11.600496: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-19 20:19:11.600566: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-19 20:19:11.600820: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmph5nsxh41
2023-01-19 20:19:11.602126: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-01-19 20:19:11.602156: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmph5nsxh41
2023-01-19 20:19:11.607410: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-01-19 20:19:11.659559: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmph5nsxh41
2023-01-19 20:19:11.672172: I tensorflow/cc/saved_model/loader.cc:305] SavedModel